# Import libraries

In [1]:
from datetime import datetime,timedelta,date
from time import time
import requests
import pandas as pd
from datetime import date,datetime
import json
from ta.volatility import BollingerBands
from ta import add_all_ta_features

### Setting start(date0) and end(date1) of time period

In [3]:
date0,date1 = datetime(2016,1,1) ,datetime.now() - timedelta(days=1)

days = (date1 - date0).days

### Creating the DataFrame for getting informations

In [4]:
DataFrame = pd.DataFrame(columns = ["time","low","high","open","close","volume","date"])

### Calculating time based on 300 request limitation on coinbase api

In [5]:
dates = []
d0 = date0
d1 = date1
date =[]
for i in range(1,(days//300)+ 1):
    
    d1 = d0 + timedelta(days=300)
    date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
    dates.append(date)
    d0 = d1

d1 = d0 + timedelta(days= days % 300)
date = [d0.strftime("%Y-%m-%d"),d1.strftime("%Y-%m-%d")]
dates.append(date)

## Fetching information from Coinbase API

In [ ]:
#API_MAIN_LINK
url = "https://api.exchange.coinbase.com"

#Currency name to scrap price 
sym = 'BTC-USD'

#start & ending time of period with size of candle
for i in dates:
    parameters = {
        'start':i[0],
        'end':i[1],
        'granularity':'86400'
    }
    data = requests.get(f"{url}/products/{sym}/candles",params=parameters,headers={'Accepts':'application/json'})
    df = pd.DataFrame(data.json(),columns = ["time","low","high","open","close","volume"])
    df["date"] = pd.to_datetime(df["time"],unit='s')
    DataFrame = DataFrame.append(df)


## Sorting the DataFrame based on Date

In [6]:
DataFrame.sort_values(by='date',inplace=True)

### Removing duplicates from Dataframe and ReIndexing the Dataframe

In [7]:
DataFrame.drop_duplicates(inplace=True)

DataFrame.reset_index(inplace=True)

### Remove Extra Indexation

In [8]:
DataFrame.drop('index',axis=1,inplace=True)

## Converting Data Frame into a CSV file

In [ ]:
DataFrame.to_csv('Prices.csv')